# Multiple networks

So far we've been querying a single model. We'll extend our database schema to
allow multiple models to be stored and queried.

## The models

We'll take our tried-and-trusted function from earlier notebooks, and train 2
different networks on it.

In [27]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import utils.duckdb as db
import utils.nn as nn
import pandas as pd

torch.manual_seed(223)

def f(x):
    if x < 0:
        return 0
    elif 0 <= x < 5:
        return x
    elif 5 <= x < 10:
        return 10-x
    else:
        return 0

x_train = np.linspace(-5, 15, 1000)
y_train = np.array([f(x) for x in x_train])

model1 = nn.ReLUFNN(input_size=1, hidden_size=2, num_hidden_layers=2, output_size=1)
nn.train(model1, x_train, y_train, save_path="models/eval_multiple_networks1.pt")

model2 = nn.ReLUFNN(input_size=1, hidden_size=3, num_hidden_layers=3, output_size=1)
nn.train(model2, x_train, y_train, save_path="models/eval_multiple_networks2.pt")

## The database

We'll construct our database as usual, but include an extra column to specify
which model the nodes/edges belong to.

First, we reset the database to make sure we start from a clean slate.

In [28]:
db.con.execute("DROP TABLE IF EXISTS edge")
db.con.execute("DROP TABLE IF EXISTS node")
db.con.execute("DROP TABLE IF EXISTS model")

db.con.execute("DROP SEQUENCE IF EXISTS seq_node")
db.con.execute("CREATE SEQUENCE seq_node START 1")

db.con.execute("DROP SEQUENCE IF EXISTS seq_model")
db.con.execute("CREATE SEQUENCE seq_model START 1")

Now we'll create a table to hold our model metadata. In this case, we simply
keep the name of the model.

In [29]:
db.con.execute(
    """
    CREATE TABLE model(
        id INTEGER PRIMARY KEY DEFAULT NEXTVAL('seq_model'),
        name TEXT
    )
    """
)

And then we'll create the node/edge tables, with the extra column to reference
the model.

In [30]:
db.con.execute(
    """
    CREATE TABLE node(
        id INTEGER PRIMARY KEY DEFAULT NEXTVAL('seq_node'),
        model_id INTEGER,
        bias REAL,
        name TEXT,
        FOREIGN KEY (model_id) REFERENCES model(id)
    )"""
)
db.con.execute(
    """
    CREATE TABLE edge(
        model_id INTEGER,
        src INTEGER,
        dst INTEGER,
        weight REAL,
        FOREIGN KEY (model_id) REFERENCES model(id),
        FOREIGN KEY (src) REFERENCES node(id),
        FOREIGN KEY (dst) REFERENCES node(id),
    )"""
)

## Inserting the models

To insert the models into the database, we'll recycle our [utility
code](./utils/duckdb.py), but make sure to add the `model_id` fields.

In [31]:
id = 0

def load_model_into_db(model, name):
    state_dict = model.state_dict()

    # Insert the model
    result = db.con.execute(
        "INSERT INTO model (name) VALUES ($name) RETURNING (id)",
        {"name": name}
    )

    (model_id,) = result.fetchone()
    node_ids = [[]]

    def nodes(node_ids):
        global id
        # Retrieves the input x weights matrix
        input_weights = list(state_dict.items())[0][1].tolist()
        num_input_nodes = len(input_weights[0])

        for i in range(0, num_input_nodes):
            id += 1
            yield [id, model_id, 0, f"input.{i}"]
            node_ids[0].append(id)

        layer = 0
        # In the first pass, insert all nodes with their biases
        for name, values in state_dict.items():
            # state_dict alternates between weight and bias tensors.
            if not "bias" in name:
                continue

            node_ids.append([])

            layer += 1
            for i, bias in enumerate(values.tolist()):
                id += 1
                yield [id, model_id, bias, f"{name}.{i}"]
                node_ids[layer].append(id)

    def edges(node_ids):
        # In the second pass, insert all edges and their weights. This assumes a fully
        # connected network.
        layer = 0
        for name, values in state_dict.items():
            # state_dict alternates between weight and bias tensors.
            if not "weight" in name:
                continue

            # Each weight tensor has a list for each node in the next layer. The
            # elements of this list correspond to the nodes of the current layer.
            weight_tensor = values.tolist()
            for from_index, from_node in enumerate(node_ids[layer]):
                for to_index, to_node in enumerate(node_ids[layer + 1]):
                    weight = weight_tensor[to_index][from_index]
                    yield [model_id, from_node, to_node, weight]

            layer += 1

    db.batch_insert(nodes(node_ids), "node")
    db.batch_insert(edges(node_ids), "edge")

    db.con.execute(f"EXPORT DATABASE 'dbs/multiple_networks.db'")

In [32]:
load_model_into_db(model1, "Model 1")
load_model_into_db(model2, "Model 2")

## Querying the database

We can now do queries across multiple models. As a basic example, let's run the
`eval` query for all models in the database for a given input. We'll give the
full query here, with comments for our adaptations.

```sql
WITH RECURSIVE input_values_single AS (
    -- We do a hardcoded query with 5 as the input value, but this could come 
    -- from anywhere.
    SELECT 0 AS input_set_id, 1 AS input_node_idx, 5 AS input_value
),
-- We make sure we have a separate input_value entry for each model in the
-- database by doing a cross join.
input_values AS (
    SELECT m.id AS model_id, input_set_id, input_node_idx, input_value
    FROM input_values_single
    CROSS JOIN model m
),
input_nodes AS (
    SELECT
        model_id,
        id,
        -- We add a PARTITION BY to make sure the row number increments
        -- separately per model.
        ROW_NUMBER() OVER (PARTITION BY model_id ORDER BY id) AS input_node_idx
    FROM node
    WHERE id NOT IN
    (SELECT dst FROM edge)
),
output_nodes AS (
    SELECT model_id, id
    FROM node
    WHERE id NOT IN
    (SELECT src FROM edge)
),
tx AS (
    SELECT
        -- Add the model ID for disambiguation.
        i.model_id,
        v.input_set_id AS input_set_id,
        GREATEST(
            0,
            n.bias + SUM(e.weight * v.input_value)
        ) AS value,
        e.dst AS id
    FROM edge e
    JOIN input_nodes i ON i.id = e.src
    JOIN node n ON e.dst = n.id
    -- Join on the correct input value.
    JOIN input_values v ON i.input_node_idx = v.input_node_idx AND v.model_id = i.model_id
    GROUP BY i.model_id, e.dst, n.bias, v.input_set_id

    UNION ALL

    SELECT
        tx.model_id,
        tx.input_set_id AS input_set_id,
        GREATEST(
            0,
            n.bias + SUM(e.weight * tx.value)
        ) AS value,
        e.dst AS id
    FROM edge e
    JOIN tx ON tx.id = e.src AND tx.model_id = e.model_id
    JOIN node n ON e.dst = n.id
    GROUP BY tx.model_id, e.dst, n.bias, tx.input_set_id
),
t_out AS (
    SELECT
        -- And add the model ID to the output as well, so we know which model
        -- gave each value.
        tx.model_id,
        tx.input_set_id AS input_set_id,
        n.bias + SUM(e.weight * tx.value) AS value,
        e.dst AS id
    FROM edge e
    JOIN output_nodes o ON e.dst = o.id
    JOIN node n ON o.id = n.id
    JOIN tx ON tx.id = e.src AND tx.model_id = e.model_id
    GROUP BY tx.model_id, e.dst, n.bias, tx.input_set_id
)
-- Include some model metadata
SELECT m.name, t.value AS output_value
FROM model m
JOIN t_out t ON t.model_id = m.id;
```

This query is the same as the one in
[eval_recursive_multiple_models.sql](./queries/eval_recursive_multiple_models.sql).
Let's execute it and compare the results.

In [35]:
with open('queries/eval_recursive_multiple_models.sql') as file:
    query = file.read()

db.con.sql(query)

┌─────────┬────────────────────┐
│  name   │    output_value    │
│ varchar │       double       │
├─────────┼────────────────────┤
│ Model 1 │   4.99999148441685 │
│ Model 2 │ 1.2487479448318481 │
└─────────┴────────────────────┘

In [34]:
model1.eval()
model2.eval()

print(model1(torch.tensor([5.0])))
print(model2(torch.tensor([5.0])))

tensor([5.0000], grad_fn=<ViewBackward0>)
tensor([1.2487], grad_fn=<ViewBackward0>)


The results are correct. This simple eval query could already be useful to see
which models have the best results for a given input.